<a href="https://colab.research.google.com/github/yongjun-l/ADHD-GNN/blob/master/models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GNN, CNN Models with Cross Validation

In [ ]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')
nb_path = '/content/notebooks'
os.symlink('/content/drive/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0,nb_path)


!pip install --upgrade pandas
!pip install torch_scatter -f https://data.pyg.org/whl/torch-1.13.0+cu116.html
!pip install torch_sparse -f https://data.pyg.org/whl/torch-1.13.0+cu116.html
!pip install torch_geometric

In [1]:
import os
import random
import time
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold

import torch
from torch import tensor, optim, nn
import torch.nn.functional as F

from torch_geometric.nn import GCNConv, SAGEConv, Linear, global_mean_pool
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader as CNNLoader
from torch.nn.functional import normalize

#from torch_geometric_temporal.nn.recurrent import DCRNN

In [2]:
proj_dir  = '/content/drive/MyDrive/3_Research_Related/ADHD_Research_Google_Colab'

# for GNN models with mutual information adj matrix
mi_dir_adhd   = proj_dir + '/DATA/MI_TABLE/mi_adhd.npy'
mi_dir_control   = proj_dir + '/DATA/MI_TABLE/mi_control.npy'

# for GNN models with correlation adj matrix
corr_dir_adhd   = proj_dir + '/DATA/CORR_TABLE/corr_adhd.npy'
corr_dir_control   = proj_dir + '/DATA/CORR_TABLE/corr_control.npy'

# for CNN model 
mi_dir_adhd_overlap = proj_dir + '/DATA/MI_TABLE/mi_adhd_overlap.npy'
mi_dir_control_overlap = proj_dir + '/DATA/MI_TABLE/mi_control_overlap.npy'

# save accuracy and trained models
result_dir = proj_dir + '/DATA/RESULTS'
model_dir = proj_dir + '/DATA/MODELS'

epoch_adhd_dir = proj_dir + '/DATA/MI_TABLE/num_epoch_ADHD.npy'
epoch_control_dir = proj_dir + '/DATA/MI_TABLE/num_epoch_CONTROL.npy'

### SAGE

In [3]:
class SAGE(torch.nn.Module):
    def __init__(self, hidden_channels, k_hop):
        super(SAGE, self).__init__()
        self.k_hop = k_hop
        #torch.manual_seed(12345)
        self.conv1 = SAGEConv(data.num_node_features, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, 2)

    def forward(self, x, edge_index, batch):
        for hop in range(self.k_hop):          
            if hop==0:
                x = self.conv1(x, edge_index)
                x = x.relu()
            
            else:
                x = self.conv2(x, edge_index)
                x = x.relu()
            
        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]
        
        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        return x

### GCN

In [4]:
class GCN(torch.nn.Module):
    def __init__(self, hidden_channels, k_hop):
        super(GCN, self).__init__()
        self.k_hop = k_hop
        torch.manual_seed(12345)
        self.conv1 = GCNConv(data.num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, 2)
        
    def forward(self, x, edge_index, batch):
        for hop in range(self.k_hop):          
            if hop==0:
                x = self.conv1(x, edge_index)
                x = x.relu()
            else:
                x = self.conv2(x, edge_index)
                x = x.relu()
        
        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]
        
        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        return x


### DIFF

```
class DIFF(torch.nn.Module):
    def __init__(self, hidden_channels,K):
        super(DIFF, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = DCRNN(data.num_node_features, hidden_channels, K)
        self.conv2 = DCRNN(hidden_channels, hidden_channels, K)
        self.lin = Linear(hidden_channels, 2)

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = x.relu()

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x
```

### CNN

In [5]:
def weight_init(m):
    if type(m) == nn.Conv2d:
        nn.init.normal_(m.weight)

class CNN(nn.Module):
    def __init__(self, n_ch=4):
        super(CNN, self).__init__()
        self.n_ch = n_ch
        
        self.part_one = nn.Sequential(
            nn.Conv2d(in_channels=n_ch, out_channels=16, kernel_size=(3,3), padding=1),
            nn.MaxPool2d(kernel_size=(3,3)),
            nn.Dropout(),
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3,3), padding=1))
        
        self.part_two_a = nn.Sequential(nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(3,3), padding=1))
        
        self.part_three_1 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(1,1), padding=1),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3,3), padding=1)
        )
        
        self.maxpool = nn.MaxPool2d(kernel_size=(3,3))
        self.conv1 = nn.Conv2d(in_channels=128, out_channels=8, kernel_size=(3,3), padding=1)
        self.conv2 = nn.Conv2d(in_channels=128, out_channels=8, kernel_size=(3,3), padding=1)
        self.conv3 = nn.Conv2d(in_channels=128, out_channels=8, kernel_size=(3,3), padding=1)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=8, kernel_size=(3,3), padding=1)
        
        self.fc = nn.Linear(in_features=32, out_features=2)
        self.softmax = nn.Softmax(dim=1)
        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()
        self.bn = nn.BatchNorm2d(8)
        
        self.part_one.apply(weight_init)
        self.part_two_a.apply(weight_init)
        self.part_three_1.apply(weight_init)
        nn.init.normal_(self.conv1.weight)
        nn.init.normal_(self.conv2.weight)
        nn.init.normal_(self.conv3.weight)
        nn.init.normal_(self.conv4.weight)
        nn.init.normal_(self.fc.weight)
        
    def part_three(self, x):
        
        x = self.part_three_1(x)
        x1 = self.maxpool(x)
        x1 = self.conv1(x1)
        
        x1 = self.bn(x1)
        x1 = nn.functional.max_pool2d(input=x1, kernel_size=x1.shape[2:])
        x1 = self.relu(x1)
        
        x2 = self.maxpool(x)
        x2 = self.conv2(x2)
        x2 = self.bn(x2)
        x2 = nn.functional.avg_pool2d(input=x2, kernel_size=x2.shape[2:])
        x2 = self.tanh(x2)
        
        x3 = self.maxpool(x)
        x3 = self.conv3(x3)
        x3 = self.bn(x3)
        x3 = nn.functional.max_pool2d(input=x3, kernel_size=x3.shape[2:])
        x3 = self.relu(x3)
        
        x4 = self.maxpool(x)
        x4 = self.conv4(x4)
        x4 = self.bn(x4)
        x4 = nn.functional.avg_pool2d(input=x4, kernel_size=x4.shape[2:])
        x4 = self.tanh(x4)
        
        x = torch.cat((x1,x2,x3,x4),1)
        x = torch.squeeze(x)
        return x
        
    def forward(self, x):
        x = self.part_one(x)
        x = self.part_two_a(x)
        x = self.part_three(x)
        x = self.fc(x)
        x = self.softmax(x)
        return x

## Create Dataset from MI tables

In [6]:
def getGraph(mi_table, y):
    """
    Input: Adjacency table with shape (epoch, channels, channels). dtype: np.array / y (label) value
    output: List that contains pyG graph data objects for each MI table.
    """
    dataset_graph=[]
    (epochs, channels, channels) = mi_table.shape # Get number of epochs and channels

    for epoch in range(epochs):
        edges_np = np.array([[0],[0]]) # Initialize edges matrix
        for row in range(channels):
            for col in range(channels):
                edge = np.array([[row],[col]]) # define fully connected edge matrix of shape (2x400)
                edges_np = np.concatenate((edges_np,edge),axis=1)
                
                # our data is unweighted
                #weight = np.array([[ADHD_mi[epoch,row,col]]])
                #weights_np = np.concatenate((weights_np, weight),axis=0)

        edges_np = edges_np[:,1:]
        edges = tensor(edges_np, dtype=torch.long)
        
        # data types are required by the loss function
        y = torch.tensor([y], dtype=torch.int64) 
        x = torch.tensor(mi_table[epoch,:,:], dtype=torch.float) # entire MI table is considered as graph data. 

        graph = Data(x=x, edge_index=edges, y=y) # Graph data stucture
        dataset_graph.append(graph)
    return dataset_graph

### Graph - Training/Validation
This dataset will be split into training set and validation set

In [7]:
#  Load mutual information matrices
ADHD_mi = np.load(mi_dir_adhd)
CONTROL_mi = np.load(mi_dir_control)

ADHD_corr = np.load(corr_dir_adhd)
CONTROL_corr = np.load(corr_dir_control)

#  Construct graph from ADHD, CONTROL groups
adhd_train_val_graph = getGraph(ADHD_mi, y=1)
control_train_val_graph = getGraph(CONTROL_mi, y=0)

adhd_train_val_graph_corr = getGraph(ADHD_corr, y=1)
control_train_val_graph_corr = getGraph(CONTROL_corr, y=0)

#  Combine and shuffle
dataset_graph = adhd_train_val_graph + control_train_val_graph
random.shuffle(dataset_graph)


print("MI table shape: ",ADHD_mi.shape, "(epochs, channels, temp)")
print("MI table shape: ",CONTROL_mi.shape, "(epochs, channels, temp)")
print("# of graphs: ",len(dataset_graph))

MI table shape:  (2231, 20, 20) (epochs, channels, temp)
MI table shape:  (1757, 20, 20) (epochs, channels, temp)
# of graphs:  3988


In [ ]:
#  Get number of epochs for each patient.
epo_per_adhd = np.load(epoch_adhd_dir)
epo_per_control = np.load(epoch_control_dir)

### Graph - Test
Test set is created by taking the average value of all mutual information tables for each patient.

In [8]:


#  For each patient, find mean value of all MI tables
#  Then create new test datapoint.
adhd_test_mi = np.zeros((61,20,20))
n=0
for i, num_epo in enumerate(epo_per_adhd):
    num_epo = int(num_epo)
    adhd_test_mi[i, :, :]= np.mean(ADHD_mi[n:n+num_epo, : , : ])
    n+=num_epo

# same procedure for control group
control_test_mi = np.zeros((60,20,20))
n=0
for i, num_epo in enumerate(epo_per_control):
    num_epo = int(num_epo)
    control_test_mi[i, :, :]= np.mean(CONTROL_mi[n:n+num_epo, : , : ])
    n+=num_epo

# getGraph function to turn this into a pyG graph data format    
adhd_test_graph = getGraph(ADHD_mi, y=1)
control_test_graph = getGraph(CONTROL_mi, y=0)

test_graph = adhd_test_graph + control_test_graph
# very important to shuffle. Unshuffled data does not learn. 
random.shuffle(test_graph)

This is the summary of graph data objects.

In [9]:
data = dataset_graph[300]
print(data)
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')
print(f'Number of features: {data.num_node_features}')

Data(x=[20, 20], edge_index=[2, 400], y=[1])
Number of nodes: 20
Number of edges: 400
Has isolated nodes: False
Has self-loops: True
Is undirected: True
Number of features: 20


### Image - Training/Validation

In [10]:
ADHD_mi_overlap = np.load(mi_dir_adhd_overlap)
CONTROL_mi_overlap = np.load(mi_dir_control_overlap)

n_ch = 4 # Motivated by R, G, B, Alpha channels
(ADHD_epochs, channels, channels) = ADHD_mi_overlap.shape
(CONTROL_epochs, channels, channels) = CONTROL_mi_overlap.shape

#  Number of images
num_img_ADHD = int(ADHD_epochs/n_ch)
num_img_CONTROL = int(CONTROL_epochs/n_ch)
n_img = num_img_ADHD+num_img_CONTROL

#  Target dataset dimension
img_data = np.zeros((n_img, n_ch, channels, channels))
label = np.zeros(n_img)

# select every 4 MI tables and assign it to img_data. This simply raises ADHD_mi_overlap's dimension by 1. 
for img in range(num_img_ADHD):
    img_data[img, :, :, :] = ADHD_mi_overlap[n_ch*img:n_ch*(img+1), :, :]
    label[img] = 1
for img in range(num_img_CONTROL):
    img_data[num_img_ADHD+img, :, :, :] = CONTROL_mi_overlap[n_ch*img : n_ch*(img+1), :, :]
    label[num_img_ADHD+img] = 0

# just like any other image dataset, all values are normalized to values between 0 and 1.
for img in range(n_img):
    for ch in range(n_ch):
        img_data[img, ch, :, :] = (img_data[img, ch, :, :]) / (np.max(img_data[img, ch, :, :]))

# TensorDataset class does not have in-built shuffle function. 
# list of integers upt to 995 is shuffled and used as an index to shuffle label and img_data.
rand_idx = np.arange(996)
random.shuffle(rand_idx)
img_data = img_data[rand_idx, : , : , :]
label = label[rand_idx]        

img_data = torch.Tensor(img_data)
label = torch.Tensor(label)
label = label.long() # loss function requires this
dataset_image = TensorDataset(img_data, label) #  Dataset class construction

print('Number of ADHD images',num_img_ADHD)
print('Number of CONTROL images',num_img_CONTROL)
print('Total images',n_img)
print('Train-Validation Image Dataset Shape',img_data.shape)

Number of ADHD images 557
Number of CONTROL images 439
Total images 996
Train-Validation Image Dataset Shape torch.Size([996, 4, 29, 29])


### Image Dataset - Test

In [11]:
epo_per_adhd = np.load(epoch_adhd_dir)
epo_per_control = np.load(epoch_control_dir)

adhd_test_img = np.zeros((61,4,29,29))
control_test_img = np.zeros((60,4,29,29))
test_label = np.zeros(121)

# since image data is 3 dimensional, there were several ways to average image for patient. 
n=0
for i, n_epo_patient in enumerate(epo_per_adhd):
    n_im_patient = int(num_epo/n_ch)
    im_patient = np.zeros((n_im_patient, 4, 29, 29))
    for j in range(n_im_patient):
        im_patient[j , : , : , :] = ADHD_mi_overlap[n+j*4 : n+(j+1)*4] # I found the images that would be in img_data.
    adhd_test_img[i, :, :, :]= np.mean(im_patient, axis=0) # then averaged the images element-wise.
    test_label[i]=1
    n+=num_epo

n=0
for i, n_epo_patient in enumerate(epo_per_control):
    n_im_patient = int(num_epo/n_ch)
    im_patient = np.zeros((n_im_patient, 4, 29, 29))
    for j in range(n_im_patient):
        im_patient[j , : , : , :] = CONTROL_mi_overlap[n+j*4 : n+(j+1)*4]
    control_test_img[i, :, :, :]= np.mean(im_patient, axis=0)
    n+=num_epo

test_img = np.concatenate((adhd_test_img, control_test_img), axis = 0)

# test set is shuffled in the same manner
rand_idx = np.arange(121)
random.shuffle(rand_idx)
test_img = test_img[rand_idx, : , : , :]
test_label = test_label[rand_idx]

test_img = torch.Tensor(test_img)
test_label = torch.Tensor(test_label)
test_label = test_label.long()
test_dataset_img = TensorDataset(test_img, test_label)

## Main Model Training and Testing with K-fold

### Train / Test Functions

In [12]:
def train(model, loader, loss_fn, optimizer):
    model.train()

    for data in loader:  # Iterate batches
        out = model(data.x, data.edge_index, data.batch)  # forward pass
        loss = loss_fn(out, data.y) # loss
        loss.backward()  # gradient
        optimizer.step()  # update weights
        optimizer.zero_grad()  # clear gradients
    return

def test(model, loader, dataset):    
    correct = 0
    for data in loader:  
        out = model(data.x, data.edge_index, data.batch) 
        pred = out.argmax(dim=1)  # make prediction based on returned softmax values
        correct += int((pred == data.y).sum()) # count correct predictions
    return correct / len(dataset)

# since CNN model does not have data object like GNN, train and test functions were implemented separately.
def train_cnn(model, loader, loss_fn, optimizer):
    correct=0
    for i, data in enumerate(loader):
        inputs, labels = data
        if inputs.shape[0] ==1:  # k-fold would randomly return a fold size that would result in 1 mod batch_size. This was causing dimention error. 
            return
        pred = model(inputs)
        loss = loss_fn(pred, labels)
        loss.backward()
        
        optimizer.step()
        optimizer.zero_grad()
        
        pred_argmax = pred.argmax(dim=1) 
        correct += int((pred_argmax == labels).sum())
        
def test_cnn(model, loader, dataset):  
    correct = 0
    running_loss = 0.0
    for i, data in enumerate(loader): 
        inputs, labels = data
        if inputs.shape[0] ==1:
            break
        pred = model(inputs)
        pred_argmax = pred.argmax(dim=1)  
        correct += int((pred_argmax == labels).sum())
        
    return correct / len(dataset) 

### Main Function

In [16]:
def main(model_name, dataset, test_dataset, model_dir, result_dir, k_hop=1, k_fold=10, n_epo=150, h_ch = 20, lr=0.0001, save=True):
    start = time.time()
    
    
    # Implement k-fold cross validation
    kf = KFold(n_splits=k_fold, shuffle=True)
    
    # For each fold
    for fold, (train_index, valid_index) in enumerate(kf.split(dataset)):

        # Define model, optimizer, and loss function
        if model_name == 'SAGE':
            model = SAGE(hidden_channels=h_ch, k_hop=k_hop)
        elif model_name == 'GCN':
            model = GCN(hidden_channels=h_ch, k_hop=k_hop)
        #elif model_name == 'DIFF':
        #    model = DIFF(hidden_channels=h_ch, K=k_hop)
        elif model_name =='CNN':
            model = CNN()
        else:
            print('Error: model not defined')
            return
        
        #opt = torch.optim.Adam(model.parameters(), lr=0.01)
        opt = optim.NAdam(model.parameters(), lr=lr, betas = (0.9,0.999), momentum_decay=0.004)
        loss_fnc = torch.nn.CrossEntropyLoss()
        
        list_train_acc = []
        list_valid_acc = []
        list_test_acc = []
        
        # Split train, test set and define dataloader
        train_dataset = [dataset[i] for i in train_index]
        valid_dataset = [dataset[i] for i in valid_index]
        
        if model_name =='CNN':
            train_loader = CNNLoader(train_dataset, batch_size=128, shuffle=False)
            valid_loader = CNNLoader(valid_dataset, batch_size=32, shuffle=False)
            test_loader = CNNLoader(test_dataset, batch_size=32, shuffle=False)

        else:
            train_loader = DataLoader(train_dataset, batch_size=128, shuffle=False)
            valid_loader = DataLoader(valid_dataset, batch_size=128, shuffle=False)
            test_loader = DataLoader(test_dataset, batch_size=121, shuffle=False)  # whole set
        
        # For each epoch
        for epoch in range(n_epo):
            if model_name == 'CNN':
                train_cnn(model, train_loader, loss_fnc, opt)
            else:
                train(model, train_loader, loss_fnc, opt)

            # Get accuracy for train and validation set
            if model_name =='CNN':
                train_acc = test_cnn(model, train_loader, train_dataset)
                valid_acc = test_cnn(model, valid_loader, valid_dataset)
                test_acc = test_cnn(model, test_loader, test_dataset)
            else:
                train_acc = test(model, train_loader, train_dataset)
                valid_acc = test(model, valid_loader, valid_dataset)
                test_acc = test(model, test_loader, test_dataset)

            list_train_acc.append(train_acc)
            list_valid_acc.append(valid_acc)
            list_test_acc.append(test_acc)
            
            #if epoch+1 % 1 ==0:
            print(f'Fold: {fold+1}, Epoch: {epoch+1:03d}, Train: {train_acc:.4f}, Valid: {valid_acc:.4f}, Test: {test_acc:.4f}')
    
    ####################################
    # Save the results for visualization and analysis
    ####################################
    if save==False:
      return
      
    # Turn accuracy to numpy array
    list_train_acc = np.array(list_train_acc)
    list_valid_acc = np.array(list_valid_acc)
    list_test_acc = np.array(list_test_acc)

    # Reshape results as column vector
    list_train_acc = np.reshape(list_train_acc, (-1,1))
    list_valid_acc = np.reshape(list_valid_acc, (-1,1))
    list_test_acc = np.reshape(list_test_acc, (-1,1))
    results = np.concatenate((list_train_acc,list_valid_acc,list_test_acc), axis=1)
    results = pd.DataFrame(results, columns=['Train', 'Valid', 'Test'])
    
    # Save accuracy log
    filename = result_dir+'/kfold_'
    if model_name == 'CNN':
        filename += f'{model_name}_ndam_epo_{n_epo}.csv'
    else:
        filename += f'{model_name}_k_{k_hop}_ndam_epo_{n_epo}_lr_{lr}.csv'
    results.to_csv(filename, float_format='%.3f', index=False, header=True)

    # Save model for later use
    filename_model = model_dir+'/kfold_'
    if model_name == 'CNN':
        filename_model += f'{model_name}.pth'
    else: 
        filename_model += f'{model_name}_k_{k_hop}.pth'
    torch.save(model, filename_model)

    # Retain saved model
    # This may not work for other environments due to different path names
    model1 = torch.load(filename_model)
    #test_acc = test(model1, test_loader, test_dataset)
    #print(f'Acc: {test_acc:.4f}')
    print('\nElapsed Time: ',time.time()-start)

## CNN Training

## SAGE Training with k = 1, 2, 3, 4

In [18]:
main('SAGE', dataset_graph, test_graph, model_dir, result_dir, k_hop=1, k_fold=10, n_epo=50, save=True)

Fold: 1, Epoch: 001, Train: 0.4341, Valid: 0.4937, Test: 0.4396
Fold: 1, Epoch: 002, Train: 0.4347, Valid: 0.4837, Test: 0.4398
Fold: 1, Epoch: 003, Train: 0.4327, Valid: 0.4862, Test: 0.4406
Fold: 1, Epoch: 004, Train: 0.4324, Valid: 0.4837, Test: 0.4378
Fold: 1, Epoch: 005, Train: 0.4372, Valid: 0.4887, Test: 0.4441
Fold: 1, Epoch: 006, Train: 0.4347, Valid: 0.4987, Test: 0.4418
Fold: 1, Epoch: 007, Train: 0.4427, Valid: 0.5038, Test: 0.4453
Fold: 1, Epoch: 008, Train: 0.4489, Valid: 0.4912, Test: 0.4526
Fold: 1, Epoch: 009, Train: 0.4533, Valid: 0.4837, Test: 0.4534
Fold: 1, Epoch: 010, Train: 0.4567, Valid: 0.5113, Test: 0.4677
Fold: 1, Epoch: 011, Train: 0.4815, Valid: 0.5188, Test: 0.4727
Fold: 1, Epoch: 012, Train: 0.4759, Valid: 0.5138, Test: 0.4809
Fold: 1, Epoch: 013, Train: 0.4862, Valid: 0.5163, Test: 0.4880
Fold: 1, Epoch: 014, Train: 0.5015, Valid: 0.5188, Test: 0.4935
Fold: 1, Epoch: 015, Train: 0.4909, Valid: 0.5063, Test: 0.5118
Fold: 1, Epoch: 016, Train: 0.5118, Vali

In [ ]:
main('SAGE', dataset_graph, test_graph, model_dir, result_dir, k_hop=2, k_fold=10, n_epo=50)

Fold: 1, Epoch: 001, Train: 0.4923, Valid: 0.4712, Test: 0.4957
Fold: 1, Epoch: 011, Train: 0.7216, Valid: 0.7343, Test: 0.7129
Fold: 1, Epoch: 021, Train: 0.7386, Valid: 0.7444, Test: 0.7482
Fold: 1, Epoch: 031, Train: 0.7581, Valid: 0.7769, Test: 0.7578
Fold: 1, Epoch: 041, Train: 0.7746, Valid: 0.7820, Test: 0.7706
Fold: 2, Epoch: 001, Train: 0.8066, Valid: 0.7920, Test: 0.8054
Fold: 2, Epoch: 011, Train: 0.8122, Valid: 0.8145, Test: 0.8175
Fold: 2, Epoch: 021, Train: 0.8412, Valid: 0.8195, Test: 0.8363
Fold: 2, Epoch: 031, Train: 0.8520, Valid: 0.8296, Test: 0.8538
Fold: 2, Epoch: 041, Train: 0.8704, Valid: 0.8296, Test: 0.8666
Fold: 3, Epoch: 001, Train: 0.8724, Valid: 0.8446, Test: 0.8706
Fold: 3, Epoch: 011, Train: 0.8780, Valid: 0.8471, Test: 0.8786
Fold: 3, Epoch: 021, Train: 0.8849, Valid: 0.8722, Test: 0.8804
Fold: 3, Epoch: 031, Train: 0.8913, Valid: 0.8772, Test: 0.8917
Fold: 3, Epoch: 041, Train: 0.8997, Valid: 0.8772, Test: 0.8907
Fold: 4, Epoch: 001, Train: 0.8927, Vali

In [ ]:
main('SAGE', dataset_graph, test_graph, model_dir, result_dir, k_hop=3, k_fold=10, n_epo=50)

Fold: 1, Epoch: 001, Train: 0.4461, Valid: 0.3910, Test: 0.4406
Fold: 1, Epoch: 011, Train: 0.6434, Valid: 0.6441, Test: 0.6261
Fold: 1, Epoch: 021, Train: 0.7359, Valid: 0.7343, Test: 0.7402
Fold: 1, Epoch: 031, Train: 0.7545, Valid: 0.7444, Test: 0.7603
Fold: 1, Epoch: 041, Train: 0.8208, Valid: 0.8271, Test: 0.8282
Fold: 2, Epoch: 001, Train: 0.8626, Valid: 0.8521, Test: 0.8561
Fold: 2, Epoch: 011, Train: 0.8805, Valid: 0.8596, Test: 0.8741
Fold: 2, Epoch: 021, Train: 0.8863, Valid: 0.8496, Test: 0.8847
Fold: 2, Epoch: 031, Train: 0.8947, Valid: 0.8672, Test: 0.8934
Fold: 2, Epoch: 041, Train: 0.9016, Valid: 0.8596, Test: 0.8949
Fold: 3, Epoch: 001, Train: 0.9081, Valid: 0.8772, Test: 0.9087
Fold: 3, Epoch: 011, Train: 0.9189, Valid: 0.8747, Test: 0.9193
Fold: 3, Epoch: 021, Train: 0.9195, Valid: 0.8922, Test: 0.9142
Fold: 3, Epoch: 031, Train: 0.9225, Valid: 0.8897, Test: 0.9183
Fold: 3, Epoch: 041, Train: 0.9351, Valid: 0.8872, Test: 0.9288
Fold: 4, Epoch: 001, Train: 0.9292, Vali

In [ ]:
main('SAGE', dataset_graph, test_graph, model_dir, result_dir, k_hop=4, k_fold=10, n_epo=50)

Fold: 1, Epoch: 001, Train: 0.4999, Valid: 0.5238, Test: 0.4920
Fold: 1, Epoch: 011, Train: 0.7099, Valid: 0.7168, Test: 0.7121
Fold: 1, Epoch: 021, Train: 0.7283, Valid: 0.7544, Test: 0.7309
Fold: 1, Epoch: 031, Train: 0.8041, Valid: 0.8271, Test: 0.8087
Fold: 1, Epoch: 041, Train: 0.8437, Valid: 0.8596, Test: 0.8410
Fold: 2, Epoch: 001, Train: 0.8468, Valid: 0.8521, Test: 0.8460
Fold: 2, Epoch: 011, Train: 0.8766, Valid: 0.8747, Test: 0.8774
Fold: 2, Epoch: 021, Train: 0.8899, Valid: 0.8897, Test: 0.8947
Fold: 2, Epoch: 031, Train: 0.8989, Valid: 0.8997, Test: 0.8952
Fold: 2, Epoch: 041, Train: 0.9089, Valid: 0.8972, Test: 0.9087
Fold: 3, Epoch: 001, Train: 0.9145, Valid: 0.9198, Test: 0.9155
Fold: 3, Epoch: 011, Train: 0.9178, Valid: 0.9198, Test: 0.9188
Fold: 3, Epoch: 021, Train: 0.9209, Valid: 0.9273, Test: 0.9208
Fold: 3, Epoch: 031, Train: 0.9326, Valid: 0.9148, Test: 0.9275
Fold: 3, Epoch: 041, Train: 0.9351, Valid: 0.9248, Test: 0.9338
Fold: 4, Epoch: 001, Train: 0.9348, Vali

In [ ]:
main('SAGE', dataset_graph, test_graph, model_dir, result_dir, k_hop=5, k_fold=10, n_epo=50)

Fold: 1, Epoch: 001, Train: 0.5272, Valid: 0.5414, Test: 0.5306
Fold: 1, Epoch: 011, Train: 0.6991, Valid: 0.7218, Test: 0.7009
Fold: 1, Epoch: 021, Train: 0.7492, Valid: 0.7569, Test: 0.7550
Fold: 1, Epoch: 031, Train: 0.7751, Valid: 0.7820, Test: 0.7791
Fold: 1, Epoch: 041, Train: 0.8247, Valid: 0.8221, Test: 0.8260
Fold: 2, Epoch: 001, Train: 0.8417, Valid: 0.8396, Test: 0.8430
Fold: 2, Epoch: 011, Train: 0.8649, Valid: 0.8546, Test: 0.8586
Fold: 2, Epoch: 021, Train: 0.8819, Valid: 0.8546, Test: 0.8756
Fold: 2, Epoch: 031, Train: 0.8997, Valid: 0.8947, Test: 0.9012
Fold: 2, Epoch: 041, Train: 0.9089, Valid: 0.9123, Test: 0.9107
Fold: 3, Epoch: 001, Train: 0.9170, Valid: 0.8997, Test: 0.9132
Fold: 3, Epoch: 011, Train: 0.9245, Valid: 0.8897, Test: 0.9175
Fold: 3, Epoch: 021, Train: 0.9284, Valid: 0.8972, Test: 0.9228
Fold: 3, Epoch: 031, Train: 0.9245, Valid: 0.8972, Test: 0.9203
Fold: 3, Epoch: 041, Train: 0.9250, Valid: 0.8897, Test: 0.9185
Fold: 4, Epoch: 001, Train: 0.9287, Vali

In [ ]:
main('SAGE', dataset_graph, test_graph, model_dir, result_dir, k_hop=6, k_fold=10, n_epo=50)

Fold: 1, Epoch: 001, Train: 0.5517, Valid: 0.5739, Test: 0.5429
Fold: 1, Epoch: 011, Train: 0.7373, Valid: 0.7444, Test: 0.7390
Fold: 1, Epoch: 021, Train: 0.7885, Valid: 0.7744, Test: 0.7849
Fold: 1, Epoch: 031, Train: 0.8548, Valid: 0.8296, Test: 0.8581
Fold: 1, Epoch: 041, Train: 0.8872, Valid: 0.8697, Test: 0.8879
Fold: 2, Epoch: 001, Train: 0.8980, Valid: 0.9248, Test: 0.9012
Fold: 2, Epoch: 011, Train: 0.9050, Valid: 0.9198, Test: 0.9115
Fold: 2, Epoch: 021, Train: 0.9309, Valid: 0.9449, Test: 0.9330
Fold: 2, Epoch: 031, Train: 0.9329, Valid: 0.9398, Test: 0.9333
Fold: 2, Epoch: 041, Train: 0.9379, Valid: 0.9373, Test: 0.9406
Fold: 3, Epoch: 001, Train: 0.8069, Valid: 0.8120, Test: 0.8047
Fold: 3, Epoch: 011, Train: 0.9504, Valid: 0.9223, Test: 0.9473
Fold: 3, Epoch: 021, Train: 0.9574, Valid: 0.9398, Test: 0.9544
Fold: 3, Epoch: 031, Train: 0.9479, Valid: 0.9173, Test: 0.9463
Fold: 3, Epoch: 041, Train: 0.9688, Valid: 0.9223, Test: 0.9631
Fold: 4, Epoch: 001, Train: 0.9685, Vali

In [ ]:
main('SAGE', dataset_graph, test_graph, model_dir, result_dir, k_hop=7, k_fold=10, n_epo=50)

Fold: 1, Epoch: 001, Train: 0.5472, Valid: 0.5414, Test: 0.5527
Fold: 1, Epoch: 011, Train: 0.7478, Valid: 0.7218, Test: 0.7395
Fold: 1, Epoch: 021, Train: 0.7788, Valid: 0.7519, Test: 0.7788
Fold: 1, Epoch: 031, Train: 0.7896, Valid: 0.7594, Test: 0.7849
Fold: 1, Epoch: 041, Train: 0.8286, Valid: 0.8070, Test: 0.8277
Fold: 2, Epoch: 001, Train: 0.7894, Valid: 0.7794, Test: 0.7921
Fold: 2, Epoch: 011, Train: 0.8866, Valid: 0.8647, Test: 0.8811
Fold: 2, Epoch: 021, Train: 0.8902, Valid: 0.8622, Test: 0.8892
Fold: 2, Epoch: 031, Train: 0.9075, Valid: 0.8747, Test: 0.9012
Fold: 2, Epoch: 041, Train: 0.9047, Valid: 0.8797, Test: 0.9030
Fold: 3, Epoch: 001, Train: 0.9145, Valid: 0.9273, Test: 0.9165
Fold: 3, Epoch: 011, Train: 0.9303, Valid: 0.9248, Test: 0.9305
Fold: 3, Epoch: 021, Train: 0.9356, Valid: 0.9198, Test: 0.9343
Fold: 3, Epoch: 031, Train: 0.9234, Valid: 0.9223, Test: 0.9268
Fold: 3, Epoch: 041, Train: 0.9432, Valid: 0.9323, Test: 0.9403
Fold: 4, Epoch: 001, Train: 0.9473, Vali

In [ ]:
main('SAGE', dataset_graph, test_graph, model_dir, result_dir, k_hop=8, k_fold=10, n_epo=50)

Fold: 1, Epoch: 001, Train: 0.5394, Valid: 0.5539, Test: 0.5436
Fold: 1, Epoch: 011, Train: 0.7905, Valid: 0.7845, Test: 0.7886
Fold: 1, Epoch: 021, Train: 0.8103, Valid: 0.8070, Test: 0.8094
Fold: 1, Epoch: 031, Train: 0.8390, Valid: 0.8195, Test: 0.8383
Fold: 1, Epoch: 041, Train: 0.8724, Valid: 0.8596, Test: 0.8643
Fold: 2, Epoch: 001, Train: 0.9030, Valid: 0.9073, Test: 0.9032
Fold: 2, Epoch: 011, Train: 0.9206, Valid: 0.8972, Test: 0.9215
Fold: 2, Epoch: 021, Train: 0.9117, Valid: 0.9123, Test: 0.9112
Fold: 2, Epoch: 031, Train: 0.9211, Valid: 0.9223, Test: 0.9198
Fold: 2, Epoch: 041, Train: 0.9376, Valid: 0.9248, Test: 0.9341
Fold: 3, Epoch: 001, Train: 0.9379, Valid: 0.9398, Test: 0.9406
Fold: 3, Epoch: 011, Train: 0.9390, Valid: 0.9373, Test: 0.9366
Fold: 3, Epoch: 021, Train: 0.9487, Valid: 0.9373, Test: 0.9483
Fold: 3, Epoch: 031, Train: 0.9493, Valid: 0.9424, Test: 0.9483
Fold: 3, Epoch: 041, Train: 0.9635, Valid: 0.9574, Test: 0.9596
Fold: 4, Epoch: 001, Train: 0.9607, Vali

In [ ]:
main('SAGE', dataset_graph, test_graph, model_dir, result_dir, k_hop=9, k_fold=10, n_epo=50)

Fold: 1, Epoch: 001, Train: 0.4990, Valid: 0.5138, Test: 0.4960
Fold: 1, Epoch: 011, Train: 0.7214, Valid: 0.7018, Test: 0.7227
Fold: 1, Epoch: 021, Train: 0.7852, Valid: 0.7694, Test: 0.7766
Fold: 1, Epoch: 031, Train: 0.7802, Valid: 0.7619, Test: 0.7839
Fold: 1, Epoch: 041, Train: 0.8194, Valid: 0.8271, Test: 0.8222
Fold: 2, Epoch: 001, Train: 0.8459, Valid: 0.8747, Test: 0.8490
Fold: 2, Epoch: 011, Train: 0.8532, Valid: 0.8822, Test: 0.8581
Fold: 2, Epoch: 021, Train: 0.8716, Valid: 0.8872, Test: 0.8719
Fold: 2, Epoch: 031, Train: 0.8952, Valid: 0.9223, Test: 0.8927
Fold: 2, Epoch: 041, Train: 0.9025, Valid: 0.9098, Test: 0.9010
Fold: 3, Epoch: 001, Train: 0.9072, Valid: 0.8947, Test: 0.9075
Fold: 3, Epoch: 011, Train: 0.9122, Valid: 0.9223, Test: 0.9127
Fold: 3, Epoch: 021, Train: 0.9198, Valid: 0.8922, Test: 0.9183
Fold: 3, Epoch: 031, Train: 0.9359, Valid: 0.8972, Test: 0.9328
Fold: 3, Epoch: 041, Train: 0.9423, Valid: 0.9098, Test: 0.9386
Fold: 4, Epoch: 001, Train: 0.9379, Vali

In [ ]:
main('SAGE', dataset_graph, test_graph, model_dir, result_dir, k_hop=10, k_fold=10, n_epo=50)

Fold: 1, Epoch: 001, Train: 0.5578, Valid: 0.5739, Test: 0.5594
Fold: 1, Epoch: 011, Train: 0.4670, Valid: 0.4536, Test: 0.4677
Fold: 1, Epoch: 021, Train: 0.7924, Valid: 0.8020, Test: 0.7939
Fold: 1, Epoch: 031, Train: 0.7370, Valid: 0.7093, Test: 0.7385
Fold: 1, Epoch: 041, Train: 0.8398, Valid: 0.8246, Test: 0.8360
Fold: 2, Epoch: 001, Train: 0.8632, Valid: 0.8647, Test: 0.8671
Fold: 2, Epoch: 011, Train: 0.8919, Valid: 0.8797, Test: 0.8919
Fold: 2, Epoch: 021, Train: 0.9061, Valid: 0.8897, Test: 0.9037
Fold: 2, Epoch: 031, Train: 0.9136, Valid: 0.8797, Test: 0.9125
Fold: 2, Epoch: 041, Train: 0.9248, Valid: 0.8922, Test: 0.9188
Fold: 3, Epoch: 001, Train: 0.9234, Valid: 0.9223, Test: 0.9210
Fold: 3, Epoch: 011, Train: 0.9298, Valid: 0.9123, Test: 0.9295
Fold: 3, Epoch: 021, Train: 0.9292, Valid: 0.9148, Test: 0.9270
Fold: 3, Epoch: 031, Train: 0.9384, Valid: 0.9173, Test: 0.9386
Fold: 3, Epoch: 041, Train: 0.9462, Valid: 0.9223, Test: 0.9446
Fold: 4, Epoch: 001, Train: 0.9320, Vali

## GCN Training with k = 1, 2, 3, 4

In [ ]:
main('GCN', dataset_graph, test_graph, model_dir, result_dir, k_hop=1, k_fold=10, n_epo=50)

Fold: 1, Epoch: 001, Train: 0.4466, Valid: 0.4361, Test: 0.4438
Fold: 1, Epoch: 011, Train: 0.5068, Valid: 0.5138, Test: 0.5158
Fold: 1, Epoch: 021, Train: 0.5559, Valid: 0.5815, Test: 0.5644
Fold: 1, Epoch: 031, Train: 0.5592, Valid: 0.5714, Test: 0.5634
Fold: 1, Epoch: 041, Train: 0.5626, Valid: 0.5890, Test: 0.5680
Fold: 2, Epoch: 001, Train: 0.5706, Valid: 0.5865, Test: 0.5692
Fold: 2, Epoch: 011, Train: 0.5804, Valid: 0.5990, Test: 0.5735
Fold: 2, Epoch: 021, Train: 0.5765, Valid: 0.5940, Test: 0.5810
Fold: 2, Epoch: 031, Train: 0.5779, Valid: 0.5865, Test: 0.5817
Fold: 2, Epoch: 041, Train: 0.5782, Valid: 0.6015, Test: 0.5858
Fold: 3, Epoch: 001, Train: 0.5910, Valid: 0.5915, Test: 0.5928
Fold: 3, Epoch: 011, Train: 0.5846, Valid: 0.5940, Test: 0.5965
Fold: 3, Epoch: 021, Train: 0.5971, Valid: 0.6115, Test: 0.5970
Fold: 3, Epoch: 031, Train: 0.6030, Valid: 0.6115, Test: 0.6013
Fold: 3, Epoch: 041, Train: 0.6010, Valid: 0.5990, Test: 0.6036
Fold: 4, Epoch: 001, Train: 0.6113, Vali

In [ ]:
main('GCN', dataset_graph, test_graph, model_dir, result_dir, k_hop=2, k_fold=10, n_epo=50)

Fold: 1, Epoch: 001, Train: 0.4455, Valid: 0.3810, Test: 0.4388
Fold: 1, Epoch: 011, Train: 0.5319, Valid: 0.5213, Test: 0.5374
Fold: 1, Epoch: 021, Train: 0.5458, Valid: 0.5890, Test: 0.5567
Fold: 1, Epoch: 031, Train: 0.5712, Valid: 0.6216, Test: 0.5710
Fold: 1, Epoch: 041, Train: 0.5815, Valid: 0.6341, Test: 0.5915
Fold: 2, Epoch: 001, Train: 0.5965, Valid: 0.6015, Test: 0.5933
Fold: 2, Epoch: 011, Train: 0.6007, Valid: 0.5915, Test: 0.6006
Fold: 2, Epoch: 021, Train: 0.6016, Valid: 0.6341, Test: 0.6106
Fold: 2, Epoch: 031, Train: 0.6119, Valid: 0.6165, Test: 0.6153
Fold: 2, Epoch: 041, Train: 0.6172, Valid: 0.6291, Test: 0.6181
Fold: 3, Epoch: 001, Train: 0.6342, Valid: 0.5965, Test: 0.6239
Fold: 3, Epoch: 011, Train: 0.6372, Valid: 0.6140, Test: 0.6339
Fold: 3, Epoch: 021, Train: 0.6486, Valid: 0.6190, Test: 0.6452
Fold: 3, Epoch: 031, Train: 0.6598, Valid: 0.6040, Test: 0.6530
Fold: 3, Epoch: 041, Train: 0.6645, Valid: 0.6040, Test: 0.6713
Fold: 4, Epoch: 001, Train: 0.6718, Vali

In [ ]:
main('GCN', dataset_graph, test_graph, model_dir, result_dir, k_hop=3, k_fold=10, n_epo=50)

Fold: 1, Epoch: 001, Train: 0.4288, Valid: 0.4586, Test: 0.4341
Fold: 1, Epoch: 011, Train: 0.5481, Valid: 0.5614, Test: 0.5481
Fold: 1, Epoch: 021, Train: 0.5698, Valid: 0.5163, Test: 0.5634
Fold: 1, Epoch: 031, Train: 0.5709, Valid: 0.5338, Test: 0.5695
Fold: 1, Epoch: 041, Train: 0.5801, Valid: 0.5363, Test: 0.5795
Fold: 2, Epoch: 001, Train: 0.5949, Valid: 0.6316, Test: 0.6006
Fold: 2, Epoch: 011, Train: 0.6130, Valid: 0.6416, Test: 0.6158
Fold: 2, Epoch: 021, Train: 0.6330, Valid: 0.6642, Test: 0.6316
Fold: 2, Epoch: 031, Train: 0.6506, Valid: 0.6491, Test: 0.6632
Fold: 2, Epoch: 041, Train: 0.6801, Valid: 0.6867, Test: 0.6773
Fold: 3, Epoch: 001, Train: 0.6930, Valid: 0.6441, Test: 0.6958
Fold: 3, Epoch: 011, Train: 0.7030, Valid: 0.6967, Test: 0.7079
Fold: 3, Epoch: 021, Train: 0.7069, Valid: 0.6817, Test: 0.7081
Fold: 3, Epoch: 031, Train: 0.7133, Valid: 0.7143, Test: 0.7202
Fold: 3, Epoch: 041, Train: 0.7164, Valid: 0.7218, Test: 0.7239
Fold: 4, Epoch: 001, Train: 0.7233, Vali

In [ ]:
main('GCN', dataset_graph, test_graph, model_dir, result_dir, k_hop=4, k_fold=10, n_epo=50)

Fold: 1, Epoch: 001, Train: 0.4313, Valid: 0.4411, Test: 0.4328
Fold: 1, Epoch: 011, Train: 0.5447, Valid: 0.5439, Test: 0.5429
Fold: 1, Epoch: 021, Train: 0.5692, Valid: 0.5388, Test: 0.5657
Fold: 1, Epoch: 031, Train: 0.5756, Valid: 0.5614, Test: 0.5787
Fold: 1, Epoch: 041, Train: 0.5957, Valid: 0.5564, Test: 0.5945
Fold: 2, Epoch: 001, Train: 0.6038, Valid: 0.5789, Test: 0.5960
Fold: 2, Epoch: 011, Train: 0.6096, Valid: 0.5739, Test: 0.6056
Fold: 2, Epoch: 021, Train: 0.6216, Valid: 0.5815, Test: 0.6284
Fold: 2, Epoch: 031, Train: 0.6434, Valid: 0.6216, Test: 0.6459
Fold: 2, Epoch: 041, Train: 0.6698, Valid: 0.6692, Test: 0.6647
Fold: 3, Epoch: 001, Train: 0.6896, Valid: 0.7018, Test: 0.6978
Fold: 3, Epoch: 011, Train: 0.7052, Valid: 0.7393, Test: 0.7006
Fold: 3, Epoch: 021, Train: 0.7108, Valid: 0.7293, Test: 0.7011
Fold: 3, Epoch: 031, Train: 0.7122, Valid: 0.7419, Test: 0.7197
Fold: 3, Epoch: 041, Train: 0.7269, Valid: 0.7544, Test: 0.7252
Fold: 4, Epoch: 001, Train: 0.7286, Vali

In [ ]:
main('GCN', dataset_graph, test_graph, model_dir, result_dir, k_hop=5, k_fold=10, n_epo=50)

Fold: 1, Epoch: 001, Train: 0.4416, Valid: 0.4010, Test: 0.4358
Fold: 1, Epoch: 011, Train: 0.5266, Valid: 0.5614, Test: 0.5318
Fold: 1, Epoch: 021, Train: 0.5475, Valid: 0.5714, Test: 0.5559
Fold: 1, Epoch: 031, Train: 0.5600, Valid: 0.5840, Test: 0.5589
Fold: 1, Epoch: 041, Train: 0.5704, Valid: 0.6165, Test: 0.5762
Fold: 2, Epoch: 001, Train: 0.5926, Valid: 0.5664, Test: 0.5850
Fold: 2, Epoch: 011, Train: 0.6174, Valid: 0.5890, Test: 0.6116
Fold: 2, Epoch: 021, Train: 0.6425, Valid: 0.6341, Test: 0.6487
Fold: 2, Epoch: 031, Train: 0.6829, Valid: 0.6491, Test: 0.6871
Fold: 2, Epoch: 041, Train: 0.7136, Valid: 0.6466, Test: 0.7034
Fold: 3, Epoch: 001, Train: 0.7164, Valid: 0.6992, Test: 0.7141
Fold: 3, Epoch: 011, Train: 0.7233, Valid: 0.6917, Test: 0.7199
Fold: 3, Epoch: 021, Train: 0.7283, Valid: 0.7193, Test: 0.7292
Fold: 3, Epoch: 031, Train: 0.7345, Valid: 0.7544, Test: 0.7335
Fold: 3, Epoch: 041, Train: 0.7448, Valid: 0.7168, Test: 0.7437
Fold: 4, Epoch: 001, Train: 0.7481, Vali

In [ ]:
main('GCN', dataset_graph, test_graph, model_dir, result_dir, k_hop=6, k_fold=10, n_epo=50)

Fold: 1, Epoch: 001, Train: 0.4322, Valid: 0.4862, Test: 0.4368
Fold: 1, Epoch: 011, Train: 0.5383, Valid: 0.5163, Test: 0.5411
Fold: 1, Epoch: 021, Train: 0.5617, Valid: 0.4987, Test: 0.5559
Fold: 1, Epoch: 031, Train: 0.5834, Valid: 0.5088, Test: 0.5792
Fold: 1, Epoch: 041, Train: 0.6436, Valid: 0.6090, Test: 0.6422
Fold: 2, Epoch: 001, Train: 0.6810, Valid: 0.7343, Test: 0.6933
Fold: 2, Epoch: 011, Train: 0.7086, Valid: 0.7118, Test: 0.7109
Fold: 2, Epoch: 021, Train: 0.7308, Valid: 0.7268, Test: 0.7262
Fold: 2, Epoch: 031, Train: 0.7381, Valid: 0.7519, Test: 0.7365
Fold: 2, Epoch: 041, Train: 0.7386, Valid: 0.7494, Test: 0.7500
Fold: 3, Epoch: 001, Train: 0.7540, Valid: 0.7569, Test: 0.7538
Fold: 3, Epoch: 011, Train: 0.7609, Valid: 0.7569, Test: 0.7568
Fold: 3, Epoch: 021, Train: 0.7590, Valid: 0.7519, Test: 0.7563
Fold: 3, Epoch: 031, Train: 0.7568, Valid: 0.7569, Test: 0.7628
Fold: 3, Epoch: 041, Train: 0.7621, Valid: 0.7669, Test: 0.7620
Fold: 4, Epoch: 001, Train: 0.7634, Vali

In [ ]:
main('GCN', dataset_graph, test_graph, model_dir, result_dir, k_hop=7, k_fold=10, n_epo=50)

Fold: 1, Epoch: 001, Train: 0.4388, Valid: 0.4236, Test: 0.4376
Fold: 1, Epoch: 011, Train: 0.5333, Valid: 0.5288, Test: 0.5401
Fold: 1, Epoch: 021, Train: 0.5495, Valid: 0.5639, Test: 0.5539
Fold: 1, Epoch: 031, Train: 0.5595, Valid: 0.5639, Test: 0.5557
Fold: 1, Epoch: 041, Train: 0.5698, Valid: 0.5865, Test: 0.5765
Fold: 2, Epoch: 001, Train: 0.6417, Valid: 0.6591, Test: 0.6499
Fold: 2, Epoch: 011, Train: 0.7030, Valid: 0.6842, Test: 0.6923
Fold: 2, Epoch: 021, Train: 0.7303, Valid: 0.7318, Test: 0.7302
Fold: 2, Epoch: 031, Train: 0.7409, Valid: 0.7343, Test: 0.7447
Fold: 2, Epoch: 041, Train: 0.7506, Valid: 0.7093, Test: 0.7477
Fold: 3, Epoch: 001, Train: 0.7501, Valid: 0.8045, Test: 0.7503
Fold: 3, Epoch: 011, Train: 0.7551, Valid: 0.8095, Test: 0.7540
Fold: 3, Epoch: 021, Train: 0.7581, Valid: 0.8045, Test: 0.7613
Fold: 3, Epoch: 031, Train: 0.7517, Valid: 0.8045, Test: 0.7548
Fold: 3, Epoch: 041, Train: 0.7621, Valid: 0.8045, Test: 0.7691
Fold: 4, Epoch: 001, Train: 0.7726, Vali

In [ ]:
main('GCN', dataset_graph, test_graph, model_dir, result_dir, k_hop=8, k_fold=10, n_epo=50)

In [ ]:
main('GCN', dataset_graph, test_graph, model_dir, result_dir, k_hop=9, k_fold=10, n_epo=50)

In [ ]:
main('GCN', dataset_graph, test_graph, model_dir, result_dir, k_hop=10, k_fold=10, n_epo=50)

## DIFF Training with k = 1, 2, 3, 4

In [ ]:
main('DIFF', dataset_graph, test_graph, model_dir, result_dir, k_hop=1, k_fold=10, n_epo=50)

In [ ]:
main('DIFF', dataset_graph, test_graph, model_dir, result_dir, k_hop=2, k_fold=10, n_epo=50)

In [ ]:
main('DIFF', dataset_graph, test_graph, model_dir, result_dir, k_hop=3, k_fold=10, n_epo=50)

In [ ]:
main('DIFF', dataset_graph, test_graph, model_dir, result_dir, k_hop=4, k_fold=10, n_epo=50)

## CNN Training

In [ ]:
main('CNN', dataset_image, test_dataset_img, model_dir, result_dir, n_epo=50)